In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# 대기시간 설정을 위한 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from urllib.parse import quote
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta
from tqdm import tqdm

# 시스템 확인 - 파일 경로 확인
import os
# 다운로드 파일이동
import shutil

#엑셀파일 불러들이기 위한
import xlrd

#time.sleep을 위한 난수생성
import random

import pandas as pd

### 제주도 농어촌민박(펜션, 게스트하우스, 민박 등)
- 출처 - 제주데이터허브
- https://jejudatahub.net/data/view/data/956


In [2]:
df_accomo = pd.read_csv('./제주농어촌민박현황.csv')
df_accomo.head(20)

,읍면동,민박명,소재지,데이터기준일
0,한림읍,기영 민박,제주시 한림읍 한림로 345,20211123
1,한림읍,에너벨리,제주시 한림읍 한림로 356,20211123
2,한림읍,협재 빌리지,제주시 한림읍 한림로 338-1,20211123
3,한림읍,해송민박,제주시 한림읍 협재4길 7-15,20211123
4,한림읍,영심 민박,제주시 한림읍 협재4길 7-7,20211123
5,한림읍,원룸민박,제주시 한림읍 비양도길 275,20211123
6,한림읍,바다그리기,제주시 한림읍 옹포2길 16,20211123
7,한림읍,비양도 민박,제주시 한림읍 비양도길 8,20211123
8,한림읍,해조대 민박,제주시 한림읍 한림로 339,20211123
9,한림읍,이쁜새민박,제주시 한림읍 협재로 68-5,20211123


In [3]:
accomo_list = df_accomo['민박명']
accomo_list

0              기영 민박
1               에너벨리
2             협재 빌리지
3               해송민박
4              영심 민박
            ...     
2979         화랑펜션2층집
2980    김군빌리지 게스트하우스
2981           도두봉팬션
2982           바다성펜션
2983             올레뷰
Name: 민박명, Length: 2984, dtype: object

In [4]:
accomo_list1 = df_accomo[df_accomo['읍면동'] == '한림읍']['민박명']
accomo_list1

0       기영 민박
1        에너벨리
2      협재 빌리지
3        해송민박
4       영심 민박
        ...  
459    포인트 무드
460    스테이 담담
461        오닐
462     호미하우스
463       코티지
Name: 민박명, Length: 464, dtype: object

### 네이버 블로그 데이터 확인



In [12]:
# 제주 + "제주낭만펜션" 검색결과 URL
# "" 포함시 필수검색 옵션
url = 'https://search.naver.com/search.naver?where=view&sm=tab_jum&query=%EC%A0%9C%EC%A3%BC+%22%EB%82%AD%EB%A7%8C%EC%A0%9C%EC%A3%BC%ED%8E%9C%EC%85%98%22+-%EC%98%88%EC%95%BD%EC%BD%94%EB%93%9C+-%EC%98%88%EC%95%BD%EC%BD%94%EB%93%9C&tqi'

In [ ]:
result = []
cnt = 0

for accomo in tqdm(accomo_list):

    # 숙소 이름별 - 검색
    base_url = 'https://search.naver.com/search.naver?where=blog&&sm=tab_jum&query=%EC%A0%9C%EC%A3%BC+'
    accomo_name = quote(f'"{accomo}"')

    # 제외하고자 하는 검색어
    # 파트너스 - -%ED%8C%8C%ED%8A%B8%EB%84%88%EC%8A%A4
    # 예약코드 - -%EC%98%88%EC%95%BD%EC%BD%94%EB%93%9C
    delete_words = ["파트너스", "예약코드", '부동산']
    delete_words

    delete_search = ""
    for word in delete_words:    
        delete_search += f'+-{quote(word)}'

    # 검색 URL - 네이버 VIEW 중에서 BLOG / 숙소이름 / 제외검색어
    url = base_url + accomo_name + delete_search
    driver = webdriver.Chrome()
    driver.get(url)


    # 스크롤 내리기

    # len(soup.select('.detail_box')) = 30 첫페이지에 나오는 수가 30개
    # 30개 이하인 경우 분석에서 제외
    if len(driver.find_elements(By.CLASS_NAME, 'detail_box')) > 30:
        continue
    
    else :
        # 스크롤은 3번만 내리자
        for _ in range(5):
            last_height = driver.execute_script("return document.body.scrollHeight")

            # 창의 내리기 끝까지 하지않고, 중간에 멈춰서 한번은 page_down을 통해서
            driver.execute_script(f'window.scrollTo(0, {last_height - random.randint(1, 100)})')
            time.sleep(round(random.uniform(0.0, 1.0), 3))
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)     
            time.sleep(round(random.uniform(0.0, 1.2), 3))

            # 창의 높이가 달라지지 않으면 스크롤 내리기 중단
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
    
    time.sleep(round(random.uniform(0.0, 2.2), 3))
    try :        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        lis = soup.select_one('.lst_view').select('.view_wrap')
        
        title_list = []
        content_list = []
        blog_list = []
        time_list = []

        # 블로그가 10개 미만인 경우 분석에서 제외
        if len(lis) >= 10:
            for li in lis:
                try :
                    title_raw = li.select_one('.title_area').select_one('a').text.strip()
                    #title = re.sub('[-=+,/\?:^.@*\"※~ㆍ!』><‘|\(\)\[\]`\'…》\”\“\’·]', "", title_raw)
                    title_list.append(title_raw)
                except:
                    title_list = 'error'
                
                try :
                    content_raw = li.select_one('.dsc_link').text.strip()
                    #content = re.sub('[-=+,/\?:^.@*\"※~ㆍ!』><‘|\(\)\[\]`\'…》\”\“\’·]', "", content_raw)
                    content_list.append(content_raw)  
                except :
                    content_list = 'error'
                try :          
                    blog_link = li.select_one('.title_area').select_one('a')['href']
                    blog_list.append(blog_link)
                except :
                    blog_list = 'error'

                try :          
                    blog_time = li.select_one('.sub').text.strip()
                    time_list.append(blog_time)
                except :
                    time_list = 'error'
    except :
        title_list = []
        content_list = []
        blog_list = []
        time_list = []


    result.append({'name' : accomo,
                   'title' : title_list,
                   'contents' : content_list,
                   'time' : time_list,
                   'blog' : blog_list})
    cnt += 1
    time.sleep(round(random.uniform(0.0, 1.2), 3))

    print(f'{cnt}번째 "{accomo}" 블로그 링크 수집완료')

time.sleep(round(random.uniform(0.0, 4.2), 3))



In [21]:
li.select_one('.sub').text.strip()

'2019.06.25.'

In [9]:
df1 = pd.DataFrame(result)
df1['blog_num'] = df1['blog'].apply(lambda x: len(x))

df1.tail(20)

,name,title,contents,time,blog,blog_num
2964,낭만제주펜션,"[종달리 숙소 :: 낭만제주펜션 / 제주도 감성 숙소 / 제주서쪽숙소, 제주도 공항...",error,"[2022.02.22., 2022.10.12., 2023.01.05., 2022.0...",[https://blog.naver.com/honey_lulu/22265507120...,149
2965,미르게스트하우스,"[제주 혼자여행 숙소추천) 제주 공항근처 미르 게스트하우스, 2021 제주1, [제...",[제주 미르게스트하우스 위치 및 정보 - 입실 16:00 / 퇴실 10:00 - 침...,"[2023.08.29., 2022.04.13., 2022.07.04., 3주 전, ...","[https://blog.naver.com/kny2775/223191376449, ...",180
2966,몽똘스테이,"[제주 몽돌해변 오션뷰 카페 '윈드스테이', 제주 내도동 카페 - 윈드스테이/win...",[제주 몽돌해변 근처 오션뷰가 넘 예쁜 카페 윈드스테이 다녀온 후기임미다 윈드스테이...,"[2022.07.29., 2022.04.28., 2022.04.20., 2021.0...","[https://blog.naver.com/joiejo/222833263876, h...",180
2967,제주 포레스타 힐,[],[],[],[],0
2968,올레앤(N) 민박,[],[],[],[],0
2969,이호방,[],[],[],[],0
2970,쥬디스,"[[3박4일 제주여행] 흩어지고 부서져도 오래도록, 제주..., [제주 이호테우 숙...",[진짜 좋았다 우리의 제주 마지막 숙소! 쥬디스펜션 야외 개별스파가 있어서 이곳으로...,"[2023.09.25., 2023.01.12., 2023.03.14., 2023.0...","[https://blog.naver.com/ra_mmi/223221057778, h...",180
2971,쥬디스1,[],[],[],[],0
2972,아멩홈,[],[],[],[],0
2973,BREEZE ISLAND,[],[],[],[],0


In [10]:
df1.to_csv('./blog_list.csv')

In [11]:
df1[df1['title'] == 'error']

,name,title,contents,time,blog,blog_num


In [12]:
df1[df1['contents'] == 'error']

,name,title,contents,time,blog,blog_num
122,소희네 하우스,"[[2023.02.17 ~ 2023.02.19] 웃기만 했던 제주 여행기, 향기, ...",error,"[2023.02.21., 2023.07.17., 2023.02.28., 2023.0...",[https://blog.naver.com/su_in1025/223022853959...,180
137,바다아리,"[7개월아기랑 제주여행 1-2일차... 해물한상, 제주 이니스프리..., 아이와 함...",error,"[2023.04.14., 2022.07.24., 2022.07.09., 2023.1...",[https://blog.naver.com/happymay525/2230749882...,180
144,제주리조텔,"[(애월)빌레리조트+(협재)제주리조텔+(서귀포)이스턴제주호..., 제주 10일 살이...",error,"[2022.12.05., 2023.06.10., 2021.02.26., 2017.0...","[https://blog.naver.com/yeom8266/222946389243,...",109
160,집 머무는,"[유디 in 제주 - 둘 째 날♀️, [jeju 신혼여행 4일차] 제주 / 집 머무...",error,"[2023.06.22., 2023.06.30., 2023.11.01., 2023.0...","[https://blog.naver.com/uzi0120/223135621722, ...",131
193,협재45m,"[220619 제주여행, [제주/협재/독채펜션] 협재45m, 제주 협재 독채 펜션....",error,"[2022.06.20., 2022.04.03., 2022.08.11., 2021.1...",[https://blog.naver.com/pado-chida/22278123957...,180
...,...,...,...,...,...,...
2928,데이드림,"[제주일지 2편, [수영복리뷰]졸린 잭슨 데이드림/제주 해수욕장 추천(금능..., ...",error,"[2023.03.13., 2023.08.31., 2022.03.04., 2023.0...",[https://blog.naver.com/tldlsdmswl/22304259290...,180
2934,제주시크릿가든,"[제주 씨에스(seaes) 호텔&리조트-제주스러움을 만끽할 수..., 제주커플스냅 ...",error,"[2023.01.27., 2023.09.15., 2023.08.02., 2023.0...",[https://blog.naver.com/kminju611/222996775744...,80
2962,까르마,"[제주 독채 펜션 힐링거점이었던 풀빌라 까르마, 제주 풀빌라 미온수풀장있는 제주 까...",error,"[2023.06.01., 2023.01.16., 2023.11.02., 2023.1...","[https://blog.naver.com/enzywu/223117917995, h...",180
2964,낭만제주펜션,"[종달리 숙소 :: 낭만제주펜션 / 제주도 감성 숙소 / 제주서쪽숙소, 제주도 공항...",error,"[2022.02.22., 2022.10.12., 2023.01.05., 2022.0...",[https://blog.naver.com/honey_lulu/22265507120...,149


In [13]:
df1[df1['blog'] == 'error']

,name,title,contents,time,blog,blog_num


In [16]:
error_list = df1[df1['contents'] == 'error']['name']

In [23]:
# contents가 에러난 항목들 다시 크롤링

result1 = []
cnt = 0

# 숙소 이름별 - 검색
base_url = 'https://search.naver.com/search.naver?where=blog&&sm=tab_jum&query=%EC%A0%9C%EC%A3%BC+'
accomo_name = quote(f'"소희네 하우스"')

# 제외하고자 하는 검색어
# 파트너스 - -%ED%8C%8C%ED%8A%B8%EB%84%88%EC%8A%A4
# 예약코드 - -%EC%98%88%EC%95%BD%EC%BD%94%EB%93%9C
delete_words = ["파트너스", "예약코드", '부동산']
delete_words

delete_search = ""
for word in delete_words:    
    delete_search += f'+-{quote(word)}'

# 검색 URL - 네이버 VIEW 중에서 BLOG / 숙소이름 / 제외검색어
url = base_url + accomo_name + delete_search
driver = webdriver.Chrome()
driver.get(url)


# 제외하고자 하는 검색어
# 파트너스 - -%ED%8C%8C%ED%8A%B8%EB%84%88%EC%8A%A4
# 예약코드 - -%EC%98%88%EC%95%BD%EC%BD%94%EB%93%9C
delete_words = ["파트너스", "예약코드", '부동산']
delete_words

delete_search = ""
for word in delete_words:    
    delete_search += f'+-{quote(word)}'

# 검색 URL - 네이버 VIEW 중에서 BLOG / 숙소이름 / 제외검색어
url = base_url + accomo_name + delete_search
driver = webdriver.Chrome()
driver.get(url)


# 스크롤 내리기

# len(soup.select('.detail_box')) = 30 첫페이지에 나오는 수가 30개
# 30개 이하인 경우 분석에서 제외
# if len(driver.find_elements(By.CLASS_NAME, 'detail_box')) > 30:
#     continue

# else :
    # 스크롤은 3번만 내리자
for _ in range(5):
    last_height = driver.execute_script("return document.body.scrollHeight")

    # 창의 내리기 끝까지 하지않고, 중간에 멈춰서 한번은 page_down을 통해서
    driver.execute_script(f'window.scrollTo(0, {last_height - random.randint(1, 100)})')
    time.sleep(round(random.uniform(0.0, 1.0), 3))
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)     
    time.sleep(round(random.uniform(0.0, 1.2), 3))

    # 창의 높이가 달라지지 않으면 스크롤 내리기 중단
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break

time.sleep(round(random.uniform(0.0, 2.2), 3))
try :        
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    lis = soup.select_one('.lst_view').select('.view_wrap')
    
    title_list = []
    content_list = []
    blog_list = []
    time_list = []

    # 블로그가 10개 미만인 경우 분석에서 제외
    if len(lis) >= 10:
        for li in lis:
            try :
                title_raw = li.select_one('.title_area').select_one('a').text.strip()
                #title = re.sub('[-=+,/\?:^.@*\"※~ㆍ!』><‘|\(\)\[\]`\'…》\”\“\’·]', "", title_raw)
                title_list.append(title_raw)
            except:
                title_list = 'error'
            
            try :
                content_raw = li.select_one('.dsc_link').text.strip()
                #content = re.sub('[-=+,/\?:^.@*\"※~ㆍ!』><‘|\(\)\[\]`\'…》\”\“\’·]', "", content_raw)
                content_list.append(content_raw)  
            except :
                content_list = 'error'
            try :          
                blog_link = li.select_one('.title_area').select_one('a')['href']
                blog_list.append(blog_link)
            except :
                blog_list = 'error'

            try :          
                blog_time = li.select_one('.sub').text.strip()
                time_list.append(blog_time)
            except :
                time_list = 'error'
except :
    title_list = []
    content_list = []
    blog_list = []
    time_list = []


result1.append({'name' : accomo,
                'title' : title_list,
                'contents' : content_list,
                'time' : time_list,
                'blog' : blog_list})
cnt += 1
time.sleep(round(random.uniform(0.0, 1.2), 3))

print(f'{cnt}번째 "{accomo}" 블로그 링크 수집완료')

1번째 " " 블로그 링크 수집완료


In [31]:
result1[0]['contents']

'error'

In [32]:
li.select_one('.dsc_link').text.strip()

'갤러리 소희네하우스 -주소: 제주시 한림읍 협재남2길 45-22 -전화번호: ***-****-**** -입퇴실: 16:00 ~ 11:00 #한림펜션 #협재펜션 #협재펜션추천 #갤러리소희네하우스 협재해수욕장, 금능해변, 한림공원에서 차로 약 3분거리에 위치한 조용한 독채숙소입니다. 입구부터 넓직한게 눈에...'

In [85]:
result_e = []
cnt = 0

for accomo in tqdm(error_list):

    # 숙소 이름별 - 검색
    base_url = 'https://search.naver.com/search.naver?where=blog&&sm=tab_jum&query=%EC%A0%9C%EC%A3%BC+'
    accomo_name = quote(f'"{accomo}"')

    # 제외하고자 하는 검색어
    # 파트너스 - -%ED%8C%8C%ED%8A%B8%EB%84%88%EC%8A%A4
    # 예약코드 - -%EC%98%88%EC%95%BD%EC%BD%94%EB%93%9C
    delete_words = ["파트너스", "예약코드", '부동산']
    delete_words

    delete_search = ""
    for word in delete_words:    
        delete_search += f'+-{quote(word)}'

    # 검색 URL - 네이버 VIEW 중에서 BLOG / 숙소이름 / 제외검색어
    url = base_url + accomo_name + delete_search
    driver = webdriver.Chrome()
    driver.get(url)


    # 스크롤 내리기

    # len(soup.select('.detail_box')) = 30 첫페이지에 나오는 수가 30개
    # 30개 이하인 경우 분석에서 제외
    if len(driver.find_elements(By.CLASS_NAME, 'detail_box')) > 30:
        continue
    
    else :
        # 스크롤은 3번만 내리자
        for _ in range(5):
            last_height = driver.execute_script("return document.body.scrollHeight")

            # 창의 내리기 끝까지 하지않고, 중간에 멈춰서 한번은 page_down을 통해서
            driver.execute_script(f'window.scrollTo(0, {last_height - random.randint(1, 100)})')
            time.sleep(round(random.uniform(0.0, 1.0), 3))
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)     
            time.sleep(round(random.uniform(0.0, 1.2), 3))

            # 창의 높이가 달라지지 않으면 스크롤 내리기 중단
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
    
    time.sleep(round(random.uniform(0.0, 2.2), 3))
         
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    lis = soup.select_one('.view_wrap').select('.view_wrap')
    
    title_list = []
    content_list = []
    blog_list = []
    time_list = []

    # 블로그가 10개 미만인 경우 분석에서 제외
    if len(lis) >= 10:
        for li in lis:
        
            title_raw = li.select_one('.title_area').select_one('a').text.strip()
            #title = re.sub('[-=+,/\?:^.@*\"※~ㆍ!』><‘|\(\)\[\]`\'…》\”\“\’·]', "", title_raw)
            title_list.append(title_raw)
        
        
            content_raw = li.select_one('.dsc_area > a').text.strip()
            #content = re.sub('[-=+,/\?:^.@*\"※~ㆍ!』><‘|\(\)\[\]`\'…》\”\“\’·]', "", content_raw)
            content_list.append(content_raw)  
        
        
            blog_link = li.select_one('.title_area').select_one('a')['href']
            blog_list.append(blog_link)
        

        
            blog_time = li.select_one('.sub').text.strip()
            time_list.append(blog_time)
        
    result_e.append({'name' : accomo,
                   'title' : title_list,
                   'contents' : content_list,
                   'time' : time_list,
                   'blog' : blog_list})
    cnt += 1
    time.sleep(round(random.uniform(0.0, 1.2), 3))

    print(f'{cnt}번째 "{accomo}" 블로그 링크 수집완료')

time.sleep(round(random.uniform(0.0, 4.2), 3))


  1%|          | 1/149 [00:10<26:47, 10.86s/it]

1번째 "소희네 하우스" 블로그 링크 수집완료


  1%|          | 1/149 [00:18<46:03, 18.67s/it]


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=120.0.6099.131)
Stacktrace:
	GetHandleVerifier [0x00007FF6E5EF2142+3514994]
	(No symbol) [0x00007FF6E5B10CE2]
	(No symbol) [0x00007FF6E59B76AA]
	(No symbol) [0x00007FF6E5990AFD]
	(No symbol) [0x00007FF6E5A2CB1B]
	(No symbol) [0x00007FF6E5A4218F]
	(No symbol) [0x00007FF6E5A25D93]
	(No symbol) [0x00007FF6E59F4BDC]
	(No symbol) [0x00007FF6E59F5C64]
	GetHandleVerifier [0x00007FF6E5F1E16B+3695259]
	GetHandleVerifier [0x00007FF6E5F76737+4057191]
	GetHandleVerifier [0x00007FF6E5F6E4E3+4023827]
	GetHandleVerifier [0x00007FF6E5C404F9+689705]
	(No symbol) [0x00007FF6E5B1C048]
	(No symbol) [0x00007FF6E5B18044]
	(No symbol) [0x00007FF6E5B181C9]
	(No symbol) [0x00007FF6E5B088C4]
	BaseThreadInitThunk [0x00007FFFB56A7344+20]
	RtlUserThreadStart [0x00007FFFB6F426B1+33]


In [87]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
lis = soup.select_one('.view_wrap').select('.view_wrap')

[]

In [36]:
content_raw = li.select_one('.dsc_link').text.strip()
content_raw

'#제주테라로사 #테디베어뮤지엄 #초콜릿렌드 #박물관은살아있다제주 #연돈돈까스 #연돈볼카츠 #바다아리펜션 #협재해수욕장 #편의점은못참지 #오늘은어린이날 제주여행 4일차 여전히 비가 많이 오네요 제주허브독채펜션 나가기전 한컷!! 연돈 먹어보고 싶어서 12시 오픈인데...'

In [32]:
df1['title'] =df1['title'].apply(lambda x: ' '.join(x))
df1['contents'] =df1['contents'].apply(lambda x: ' '.join(x))
df1

,name,title,contents,blog,blog_num
0,기영 민박,[제주여행] 제주 마지막날 Day3 / 바다에서 하염없이 멍때리기... 제주 혼자 ...,옮겨 #기영민박 들려서 맥주 병나발 불고 육지로 돌아가기 싫음을 온몸으로 표현 돈벌...,"[https://blog.naver.com/imvaney/223121121422, ...",90
1,에너벨리,"해수욕장(동쪽주차장), 협재 에너벨리(식당), 카페노티드제주 몰아 올리는 제주 여행...",제주 #협재에너벨리 - 주소 : 제주 제주시 한림읍 한림로 356 에너벨리식당&민박...,[https://blog.naver.com/hi_7312319/22285822760...,120
2,협재 빌리지,2021.11.07(일)트랜스제주50 후 백패킹 금능해수욕장 제주협재해수욕장펜션 협...,옴기고 협재빌리지모텔민박 064- 796-4165 5만원에 방잡음 덴트 철수하러 토...,"[https://blog.naver.com/mowcho/222563341734, h...",15
3,해송민박,[19.03.02 ~ 05] DAY3~6 - 퇴사 여행. 8박 9일 제주 자유여행 ...,저녁 먹으러 주변에 나가볼까 했는데 죄다 브레이크타임 걸려서 포기 바람 개오졌던 협...,"[https://blog.naver.com/e__uni_/221542178596, ...",18
